In [24]:
## Load packages
library(readxl)
library(stringr)
library(rentrez)
library(dplyr)
library(tidyr)
library(jsonlite)
library(Biostrings)

In [5]:
## Adjust PATH
wd <- getwd()
subdir <- "/bin" # should be 'scripts' if location is ./scripts/thisfile
if (endsWith(wd, subdir)) {
    wd <- str_remove(wd, subdir)
    setwd(wd)
}
getwd()

[1] "/home/dieol22p/arabinosylation-anti-CRISPR"

In [6]:
## Functions

#' Get list entry and use NA for missing information
#'
#' Get entries from list entries and ignore missing information (e.g. NULL)
#' as well as errors (e.g. missing columns in data.frame)
#' 
get_list_entry <- function(x) {

    x <- tryCatch(
        error = function(cnd) NA, x
    )

    if (is.null(x)) {x <- NA}

    return(x)
}

In [7]:
# Define variables

## Input files 
in_tables <- 'docs/supplementary-tables.xlsx'

## Data storage
path <- "data/ncbi_phage-complete-genomes/"
dir.create(path, recursive = TRUE)
genomes <- list(
    accession = "accession.txt",
    overview = "overview.csv",
    annotation = "annotation.gtf",
    zip = "genomes.zip",
    archive = "genomes/ncbi_dataset/data/"
)
for (i in names(genomes)) {genomes[[i]] <- paste0(path, genomes[[i]])}

Warning message in dir.create(path, recursive = TRUE):
“'data/ncbi_phage-complete-genomes' already exists”


In [8]:
# Read data
tables <- list()
for (i in excel_sheets(in_tables)) {
    print(paste('Reading table', i))
    tables[[i]] <- read_excel(in_tables, sheet = i)
    }

[1] "Reading table S7_phage-genomes"
[1] "Reading table proteins"
[1] "Reading table S4_phages"
[1] "Reading table S1_bacteria"
[1] "Reading table S6_ara-hC-transferases"
[1] "Reading table S8_glucosylation-enzymes"
[1] "Reading table S9_DNA-modification-enzymes"


In [9]:
# Select phage genomes
data <- tables$`S7_phage-genomes`

## Check for missing IDs
paste('Missing accession IDs:', any(is.na(data$accession)))

[1] "Missing accession IDs: FALSE"

In [18]:
## Download phage genomes

# Query genomes using NCBI datasets CLI
message(paste("Downloading", length(readLines(genomes$accession)), "virus genomes"))
cli_call <- paste0("datasets download virus genome accession")
cli_call <- paste(c(cli_call, "--inputfile", genomes$accession, "--filename", genomes$zip, 
                    "--include annotation,biosample,cds,genome,protein"), collapse = " ")
system(cli_call)

In [19]:
## Extract data
unzip(genomes$zip, exdir = str_remove(genomes$zip, ".zip"))
ncbi <- list()
for (i in list.files(genomes$archive)) {
    j <- str_split(i, "\\.")[[1]][1]
    ncbi[[j]] <- paste0(genomes$archive, i)
}
ncbi

$annotation_report
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/annotation_report.jsonl"

$biosample_report
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/biosample_report.jsonl"

$cds
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/cds.fna"

$data_report
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/data_report.jsonl"

$dataset_catalog
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/dataset_catalog.json"

$genomic
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/genomic.fna"

$protein
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/protein.faa"

$virus_dataset
[1] "data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/virus_dataset.md"

In [20]:
## Investigate data report

# Read data report
report <- as.list(readLines(ncbi$data_report, skipNul = TRUE))
report <- lapply(report, fromJSON)

# Re-format
for (n in 1:length(report)) {
    x <- report[[n]]
    report[[n]] <- data.frame(
    accession = get_list_entry(x[["accession"]]),
    virusName = get_list_entry(x[["virus"]][["organismName"]]),
    virusClass = get_list_entry(x[["virus"]][["lineage"]][["name"]][[5]]),
    virusGenus = get_list_entry(x[["virus"]][["lineage"]][["name"]][[6]]),
    completeness = get_list_entry(x[["completeness"]]),
    geneCount = get_list_entry(x[["geneCount"]]),
    genomeSize = get_list_entry(x[["length"]]),
    geoLocation = get_list_entry(x[["location"]][["geographicLocation"]]),
    geoRegion = get_list_entry(x[["location"]][["geographicRegion"]]),
    labHost = get_list_entry(x[["labHost"]])
    )
}
report <- bind_rows(report)

# Check accession numbers
all(data$accession %in% report$accession)
all(report$accession %in% data$accession)

[1] FALSE

[1] FALSE

In [25]:
## Add additional information

# Read genomes
genome <- readDNAStringSet(ncbi$genomic)
names(genome) <- str_split(names(genome), ',',simplify=TRUE)[,1]

# Mutate report
report$key <- paste(report$accession, report$virusName)
report$genomePresent <- names(genome) %in% report$key
report$genomeIndex <- match(names(genome), report$key)

In [26]:
## Investigate report
x <- data$accession %in% report$accession
table(x)
data$accession[!x]

x
FALSE  TRUE 
    3    31 

[1] "NC_028683.1" "NC_055721.1" "NC_055727.1"

In [29]:
## Investigate report

# Dimensions
x <- table(report$genomePresent)
message(paste0("Genomes present \n", "True: ", x[[2]], ", False: ", x[[1]]))
message(paste(ncol(report), "Annotations"))

# View
rbind(head(report,3), tail(report,3))

# Check absence of NAs in important columns
message(paste("Any missing accession:", any(is.na(report$accession))))
message(paste("Any missing virusName:", any(is.na(report$virusName))))

Genomes present 
True: 20940, False: 4859

13 Annotations



,accession,virusName,virusClass,virusGenus,completeness,geneCount,genomeSize,geoLocation,geoRegion,labHost,key,genomePresent,genomeIndex
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>
1,OP073944.1,Bacteriophage sp.,NA,NA,COMPLETE,21,22014,Japan,Asia,NA,OP073944.1 Bacteriophage sp.,TRUE,4501
2,OP073957.1,Bacteriophage sp.,NA,NA,COMPLETE,15,17138,Japan,Asia,NA,OP073957.1 Bacteriophage sp.,TRUE,4502
3,OP073961.1,Bacteriophage sp.,NA,NA,COMPLETE,32,31550,Japan,Asia,NA,OP073961.1 Bacteriophage sp.,TRUE,4503
25797,EU855793.1,Listeria phage P40,Caudoviricetes,unclassified Caudoviricetes,COMPLETE,62,35638,Switzerland,Europe,NA,EU855793.1 Listeria phage P40,FALSE,NA
25798,EU861004.1,Staphylococcus phage vB_SauS-phiIPLA88,Caudoviricetes,Azeredovirinae,COMPLETE,61,42526,NA,NA,NA,EU861004.1 Staphylococcus phage vB_SauS-phiIPLA88,FALSE,NA
25799,EU861005.1,Staphylococcus phage phiSauS-IPLA35,Caudoviricetes,Triavirus,COMPLETE,62,45344,NA,NA,NA,EU861005.1 Staphylococcus phage phiSauS-IPLA35,FALSE,NA


Any missing accession: FALSE

Any missing virusName: FALSE



In [30]:
## Save report
write.table(report, genomes$overview, sep = ",")

In [31]:
## Format annotation to GTF
gtf_fields <- c("accession","gene-cds-name", "gene-cds-nuc-fasta-title","gene-cds-nuc-fasta-seq-id",
                "gene-cds-nuc-fasta-range-start","gene-cds-nuc-fasta-range-stop",
                "gene-cds-protein-fasta-accession","gene-cds-protein-fasta-seq-id","gene-cds-protein-fasta-title"
               )
gtf_fields <- paste0(gtf_fields, collapse = ",")
system_call <- paste("dataformat tsv virus-annotation --fields",gtf_fields,"--inputfile", ncbi$annotation_report, ">", genomes$annotation)
message(system_call)
system(system_call)

dataformat tsv virus-annotation --fields accession,gene-cds-name,gene-cds-nuc-fasta-title,gene-cds-nuc-fasta-seq-id,gene-cds-nuc-fasta-range-start,gene-cds-nuc-fasta-range-stop,gene-cds-protein-fasta-accession,gene-cds-protein-fasta-seq-id,gene-cds-protein-fasta-title --inputfile data/ncbi_phage-complete-genomes/genomes/ncbi_dataset/data/annotation_report.jsonl > data/ncbi_phage-complete-genomes/annotation.gtf



In [32]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 9.2 (Blue Onyx)

Matrix products: default
BLAS/LAPACK: /home/dieol22p/miniconda3/envs/arabinosylation-anti-crispr/lib/libopenblasp-r0.3.28.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Pacific/Auckland
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] Biostrings_2.70.1   GenomeInfoDb_1.38.1 XVector_0.42.0     
 [4] IRanges_2.36.0      S4Vectors_0.40.2    BiocGenerics_0.48.1
 [7] jsonlite_1.8.9      tidyr_1.3.1         